<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> ▼ Create Database Tables  </span>

In [1]:
import sqlite3
%run py_utils/sqlite_utils.ipynb

def create_tables(con, cur):    
    create_artists = """CREATE TABLE IF NOT EXISTS artists(
                        artist_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                        artist_name TEXT NOT NULL, 
                        artist_path TEXT NOT NULL,
                        created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
                        updated_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
                        UNIQUE(artist_name, artist_path)
                        )"""    
    run_query(cur, con, create_artists)
    print(">> Artists:", *cur.execute("PRAGMA table_info(artists)"), sep="\n") 
    
    
    create_albums = """CREATE TABLE IF NOT EXISTS albums(
                       album_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                       album_name TEXT NOT NULL, 
                       album_year INTEGER,
                       artist_id INTEGER, 
                       created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP, 
                       updated_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP, 
                       FOREIGN KEY (artist_id) REFERENCES artists (artist_id),               
                       UNIQUE(album_name, artist_id)
                       )"""
    run_query(cur, con, create_albums)
    print("\n>> Albums:", *cur.execute("PRAGMA table_info(albums)"), sep="\n") 
         
    create_tracks = """CREATE TABLE IF NOT EXISTS tracks(
                        track_id INTEGER PRIMARY KEY AUTOINCREMENT, 
                        track_name TEXT NOT NULL, 
                        track_lyrics TEXT, 
                        track_credits TEXT, 
                        track_path TEXT NOT NULL,
                        artist_id INTEGER, 
                        album_id INTEGER, 
                        created_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP, 
                        updated_at DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP, 
                        FOREIGN KEY (artist_id) REFERENCES artists (artist_id), 
                        FOREIGN KEY (album_id) REFERENCES albums (album_id),             
                        UNIQUE(track_name, track_path, artist_id, album_id)
                        )"""
    run_query(cur, con, create_tracks)
    print("\n>> Tracks:", *cur.execute("PRAGMA table_info(tracks)"), sep="\n") 
    
    
con, cur = connect_sqlite("../database/azlyrics.db") 
create_tables(con, cur)
con.close()


>> Artists:
(0, 'artist_id', 'INTEGER', 0, None, 1)
(1, 'artist_name', 'TEXT', 1, None, 0)
(2, 'artist_path', 'TEXT', 1, None, 0)
(3, 'created_at', 'DATETIME', 1, 'CURRENT_TIMESTAMP', 0)
(4, 'updated_at', 'DATETIME', 1, 'CURRENT_TIMESTAMP', 0)

>> Albums:
(0, 'album_id', 'INTEGER', 0, None, 1)
(1, 'album_name', 'TEXT', 1, None, 0)
(2, 'album_year', 'INTEGER', 0, None, 0)
(3, 'artist_id', 'INTEGER', 0, None, 0)
(4, 'created_at', 'DATETIME', 1, 'CURRENT_TIMESTAMP', 0)
(5, 'updated_at', 'DATETIME', 1, 'CURRENT_TIMESTAMP', 0)

>> Tracks:
(0, 'track_id', 'INTEGER', 0, None, 1)
(1, 'track_name', 'TEXT', 1, None, 0)
(2, 'track_lyrics', 'TEXT', 0, None, 0)
(3, 'track_credits', 'TEXT', 0, None, 0)
(4, 'track_path', 'TEXT', 1, None, 0)
(5, 'artist_id', 'INTEGER', 0, None, 0)
(6, 'album_id', 'INTEGER', 0, None, 0)
(7, 'created_at', 'DATETIME', 1, 'CURRENT_TIMESTAMP', 0)
(8, 'updated_at', 'DATETIME', 1, 'CURRENT_TIMESTAMP', 0)


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> ▼ Scrape all Artists & Store in SQLite DB  </span>

In [ ]:
%run crawlers/artist_scraping.ipynb
%run py_utils/scraping_utils.ipynb
%run py_utils/sqlite_utils.ipynb

def acquire_artists():
    return get_artists()
    
def store_artists(artist_data, con, cur):
    for artist in artist_data:
        try:
            query = "INSERT INTO artists (artist_name, artist_path) values ( ?, ? )"
            run_query(cur, con, query, values=(artist, artist_data[artist])) 
            print(">> {} added to database.".format(artist)) 
        except Exception as exc:
            print("! Exception: {}".format(exc))

con, cur = connect_sqlite("../database/azlyrics.db")  
artist_data = acquire_artists()  
store_artists(artist_data, con, cur)
con.close()


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> ▼ Artist Selection Interface </span>

In [16]:
import string
%run py_utils/widgets_utils.ipynb
%run py_utils/sqlite_utils.ipynb

user_selection = []
alphabets = [*string.ascii_uppercase, '#']
con, cur = connect_sqlite("../database/azlyrics.db")
all_artists = [data[0] for data in cur.execute("SELECT artist_name FROM artists")]
con.close()

def retrieve_alphabet(change):
    filter_artist = []
    selected_item = change.new    
    widget_output(output, "Alphabet {} selected".format(selected_item))  
    
    for artist in all_artists:
        if selected_item == artist[0].upper(): filter_artist.append(artist)            
        else:
            if artist[0].upper() not in alphabets and selected_item == '#': filter_artist.append(artist)    
    alphabet_dropdown.value = selected_item 
    artist_dropdown.options = filter_artist
    artist_dropdown.value = filter_artist[0]
    
def retrieve_artist(change): 
    widget_output(output, f"{change.new} selected")
    
def retrieve_data(_):    
    if artist_dropdown.value != '': user_selection.append(artist_dropdown.value)
    widget_output(output, 'Selection: • '+' • '.join(user_selection))

def clear_data(_):
    try:
        user_selection.pop()
        widget_output(output, 'Selection: '+', '.join(user_selection))        
    except Exception as exc: widget_output(output, '! Exception: '+str(exc))
    
output = create_output()
widget_output(output, "Please select an Artist:")
alphabet_dropdown = assign_dropdown(alphabets, "Alphabets", arg_function=retrieve_alphabet, observe=1)
artist_dropdown = assign_dropdown([''], "Artists", arg_function=retrieve_artist, observe=1)
dropdowns = create_gui(alphabet_dropdown, artist_dropdown, num=[0,1], wd='300', widget_type='H', gui_display=0)

select_button = create_button("Add Artist to Selection", arg_function=retrieve_data)
clear_button = create_button("Clear Last Selection", arg_function=clear_data)
buttons = create_gui(select_button, clear_button, num=[0,1], wd='300', widget_type='H', gui_display=0)
create_gui(output, dropdowns, buttons, widget_type='V', gui_display=1) 


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> ▼ Extract & Store Track Albums </span>

In [18]:
%run crawlers/album_scraping.ipynb
root_url = "https://www.azlyrics.com"
start_urls=[]; tmp_list = []

def get_urls(con, cur):
    for artist in user_selection:
        start_urls.append([*cur.execute("SELECT artist_path FROM artists WHERE artist_name=?", (artist,))][0][0])
        
def scrape_albums():
    return get_albums(start_urls)

def store_data(con, cur, artist_info):
    for artist in artist_info:
        artist_id = [*cur.execute("SELECT artist_id FROM artists WHERE artist_name=?", (artist,))][0][0]        
        for i, track in enumerate(artist_info[artist]['track_names']):            
            if artist_info[artist]['album_names'][i] not in tmp_list: 
                tmp_list.append(artist_info[artist]['album_names'][i])                
                try:
                    query = "INSERT INTO albums (album_name, album_year, artist_id) values ( ?, ?, ? )"
                    values = (artist_info[artist]['album_names'][i], artist_info[artist]['album_years'][i], artist_id)
                    run_query(cur, con, query, values=values)
                    print(">> Entries added:", values)                    
                except Exception as exc: print("! Exception: {}".format(exc))
                    
            try:
                cur.execute("SELECT album_id FROM albums WHERE album_name=?", (artist_info[artist]['album_names'][i],))
                album_id = cur.fetchall()[0][0] 
                track_path = (lambda url: '' if url == root_url else url) (artist_info[artist]['track_urls'][i])
                
                query = "INSERT INTO tracks (track_name, track_path, artist_id, album_id) values ( ?, ?, ?, ? )"
                values = (track, track_path, artist_id, album_id)
                run_query(cur, con, query, values=values) 
                print(">> Entries added:", values)                
            except Exception as exc: print("! Exception: {}".format(exc))

                
con, cur = connect_sqlite("../database/azlyrics.db")
get_urls(con, cur)
artist_info = scrape_albums()
store_data(con, cur, artist_info) 
con.close()


>> Scraping https://www.azlyrics.com/k/kasabian.html
! ValueError: invalid literal for int() with base 10: 'songs:'
>> Entries added: ('album: Kasabian', 2005, 9668)
>> Entries added: ('Club Foot', 'https://www.azlyrics.com/lyrics/kasabian/clubfoot.html', 9668, 1)
>> Entries added: ('Processed Beats', 'https://www.azlyrics.com/lyrics/kasabian/processedbeats.html', 9668, 1)
>> Entries added: ('Reason Is Treason', 'https://www.azlyrics.com/lyrics/kasabian/reasonistreason.html', 9668, 1)
>> Entries added: ('I.D.', 'https://www.azlyrics.com/lyrics/kasabian/id.html', 9668, 1)
>> Entries added: ('Orange (Interlude)(Instrumental)', '', 9668, 1)
>> Entries added: ('L.S.F. (Lost Souls Forever)', 'https://www.azlyrics.com/lyrics/kasabian/lsflostsoulsforever.html', 9668, 1)
>> Entries added: ('Running Battle', 'https://www.azlyrics.com/lyrics/kasabian/runningbattle.html', 9668, 1)
>> Entries added: ('Test Transmission', 'https://www.azlyrics.com/lyrics/kasabian/testtransmission.html', 9668, 1)
>>

>> Entries added: ('ALCHEMIST', 'https://www.azlyrics.com/lyrics/kasabian/alchemist.html', 9668, 7)
>> Entries added: ('SCRIPTVRE', 'https://www.azlyrics.com/lyrics/kasabian/scriptvre.html', 9668, 7)
>> Entries added: ('ROCKET FUEL', 'https://www.azlyrics.com/lyrics/kasabian/rocketfuel.html', 9668, 7)
>> Entries added: ('STRICTLY OLD SKOOL', 'https://www.azlyrics.com/lyrics/kasabian/strictlyoldskool.html', 9668, 7)
>> Entries added: ('ALYGATYR', 'https://www.azlyrics.com/lyrics/kasabian/alygatyr.html', 9668, 7)
>> Entries added: ('Ã¦ space(Instrumental)', '', 9668, 7)
>> Entries added: ('THE WALL', 'https://www.azlyrics.com/lyrics/kasabian/thewall.html', 9668, 7)
>> Entries added: ('T.U.E (the ultraview effect)', 'https://www.azlyrics.com/lyrics/kasabian/tuetheultravieweffect.html', 9668, 7)
>> Entries added: ('STARGAZR', 'https://www.azlyrics.com/lyrics/kasabian/stargazr.html', 9668, 7)
>> Entries added: ('CHEMICALS', 'https://www.azlyrics.com/lyrics/kasabian/chemicals.html', 9668, 7)

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> ▼ Generate Database Docs </span>

In [19]:
def generate_docs(db, docs, title):
    file = open(docs, "w+")
    file.write(title + "\n")
    table_tmp = "|{}|{}|\n| :-: | :-: |\n".format("Column", "Description")

    con, cur = connect_sqlite(db)
    cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    for table in cur.fetchall():
        table_name = table[0].upper()        
        file.write("### {}\n".format(table_name))
        file.write(table_tmp)
        
        cur.execute("SELECT * FROM {};".format(table_name))
        columns = [description[0] for description in cur.description]
        
        for column in columns:
            file.write("| {} | |\n".format(column))
    print(">> Docs generated.")
    file.close()    
    
generate_docs("../database/azlyrics.db", "../docs/AZlyrics_template.md", "# AZlyrics Database Documentation")
    

>> Docs generated.
